# Analysis of 2020 PSES

This is an example of a Jupyter notebook which allows a user to produce markdown between fragments of code. This will require a working instance of Python the pandas, numpy, and scipy modules (most likely through using the Anaconda distribution). Otherwise this document should be readable but the code will not run. Comments on the PSES will be written with brief explanations of the code.

## Setup
We begin by importing the necessary modules and the PSES file. The preferred file does not have labels but is easy to manipulate computationally. For some reason this file has an unusual encoding ("ANSI") and so we read it into Pandas to make it easy to manipulate and include an argument to ensure it is read correctly.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# Load Data Set
pse = pd.read_csv("2020-public-service-employee-survey-open-dataset-ensemble-de-donnees-ouvertes-du-sondage-aupres-.csv", encoding = "ANSI")

We are not interested in the entire PSES and so reduce it down to subsets. The variables LEVEL1ID  - LEVEL4ID include numerical values to identify the organizational levels. There are 5 natural points of comparison (each a subset of the other with the exception of the last):

* The Public Service as a whole
* Statistics Canada
* Economic Statistics
* Economy Wide Statistics
* Consumer Prices Division

The final subset is the Producer Prices Division (PPD) itself which is what we are interested in comparing to everything else. These subsets simplify working with these comparsions.

In [2]:
# PSE File
# LEVEL1ID = 8   - Statistics Canada
# LEVEL2ID = 203 - Economic Statistics Field
# LEVEL3ID = 306 - Economy-wide Statistics Branch
# LEVEL4ID = 417 - Producer Prices Division

public_service = pse[(pse.LEVEL1ID == 0) & (pse.LEVEL2ID == 0) & 
                     (pse.LEVEL3ID == 0) & (pse.LEVEL4ID == 0) & 
                     (pse.BYCOND == " ") & (pse.SURVEYR == 2020)]
statcan = pse[(pse.LEVEL1ID == 8) & (pse.LEVEL2ID == 0) &
              (pse.LEVEL3ID == 0) & (pse.LEVEL4ID == 0) &
              (pse.BYCOND == " ") & (pse.SURVEYR == 2020)]
econ_stat = pse[(pse.LEVEL1ID == 8) & (pse.LEVEL2ID == 203) &
                (pse.LEVEL3ID == 0) & (pse.LEVEL4ID == 0)]
econ_wide_stat = pse[(pse.LEVEL1ID == 8) & (pse.LEVEL2ID == 203) &
                     (pse.LEVEL3ID == 306) & (pse.LEVEL4ID == 0)]
ppd = pse[(pse.LEVEL1ID == 8) & (pse.LEVEL2ID == 203) &
          (pse.LEVEL3ID == 306) & (pse.LEVEL4ID == 417)]
cpd = pse[(pse.LEVEL1ID == 8) & (pse.LEVEL2ID == 203) &
          (pse.LEVEL3ID == 306) & (pse.LEVEL4ID == 416)]

In the final section of the setup we define functions to help with the analysis. Because this document is instructional there are more functions written here than we will actually use. Each function is documented but this is an example of breaking down a problem into smaller components and then reusing those components.

The essence of these functions is this. The mean results for each of the questions are compared using a t-test (specifically Welch's t-test which does not require the samples to be equal or the population variance for each sample to be equal though still performs reasonably well if they are). The end result is a data frame with the mean for each question from each subset, a p-value and adjusted significance level.

A few notes on the approaches:
* The fastest program is to simply run the test on the statistics directly. These statistics can be obtained from the survey using the proportions and and answer counts. This is the one the program uses.
* The alternative is to recreate the sample and run the t-test directly from stats. This is implemented but not used.
* A final choice is a bootstrap method. This is purely for illustration as the problem we are dealing with falls well within either Student's t-test (when adjusted for different sample sizes) or Welch's t-test. The biggest disadvantage of this approach is that each question requires about a minute and a half to two minutes to run.

In [3]:
# Define helper functions to work with.

# These helper functions are used in the analysis

def calc_mean_se(df, q, rm_ppd = False):
    """ Calculates the mean and standard error for a given DataFrame and
    question based on the proportion. Returns a tuple with mean, se, and obs.
    
    df -- The DataFrame for the sample
    q  -- The question"""
    
    ans_count = int(df[df.QUESTION == q].ANSCOUNT)
    
    # Condition to deal with Yes/No answers. Note, one implication of this fix
    # is that 'Agree' is mapped to positive which should be kept in mind when
    # reading questions like Q53
    if q in ["Q28", "Q53", "Q85", "Q87", "Q88", "Q89", "Q90"]:
        no_pos = int(((df[df.QUESTION == q].AGREE / 100) * 
                      ans_count).round())
        no_neg = ans_count - no_pos
    elif q == "Q54":
        print("Q54 is a special case, treat it manually.")
        return(0, 0, 1)
    else:
        no_pos = int(((df[df.QUESTION == q].MOST_POSITIVE_OR_LEAST_NEGATIVE / 100) * 
                      ans_count).round())
        no_neg = int(((df[df.QUESTION == q].MOST_NEGATIVE_OR_LEAST_POSITIVE / 100) * 
                      ans_count).round())
        
    if rm_ppd:
        prop_pos, _, ppd_n = calc_mean_se(ppd, q)
        ppd_pos = round(prop_pos * ppd_n)
        ppd_neg = ppd_n - ppd_pos
        
        no_pos -= ppd_pos
        no_neg -= ppd_neg

    total = no_pos + no_neg
    
    prop_pos = no_pos / total
    prop_neg = no_neg / total
    
    # Calculate sum of squares
    ss = (no_pos * (prop_neg ** 2)) + (no_neg * (prop_pos ** 2))
    se = np.sqrt(ss / (total - 1))
    
    return (prop_pos, se, total)
    
def ttest_means_from_stats(df1, df2, q, equal = False, subset = True):
    """ Applies a t-test to see if the means between two samples are
    statistically different without generating samples. Returns a tuple
    with two means and a p-value.
    
    df1   -- DataFrame for first sample
    df2   -- DataFrame for second sample
    q     -- Question to test samples for
    equal -- Argument for t-test, default is to assume variance is not equal
    
    Note on equality: in our application equal should = True but this is the
    safer default"""
    
    mean1, se1, nobs1 = calc_mean_se(df1, q)
    mean2, se2, nobs2 = calc_mean_se(df2, q, rm_ppd = subset)
    
    if not(mean1 or mean2):
        return (0, 0, 1)
    
    p_value = stats.ttest_ind_from_stats(mean1, se1, nobs1,
                                         mean2, se2, nobs2, 
                                         equal_var = equal)[1]
    
    return (mean1 * 100, mean2 * 100, p_value)

def make_comparisons(df1, df2, subset = True):
    """ Helper function to produce a data frame of comparisons sorted by
    p-values with a column for the Holm-Bonferonni correction.
    
    df1 -- Should be ppd, but left as a variable to subset on questions
    df2 -- The data frame to compare against"""
    
    ppd_means = []
    comp_means = []
    p_vals = []

    for question in df1.QUESTION:
        ppd_m, comp_m, p_val = ttest_means_from_stats(df1, df2, question, subset = subset)
        ppd_means.append(ppd_m)
        comp_means.append(comp_m)
        p_vals.append(p_val)

    ppd_df_dict = {"question": df1.QUESTION,
                   "ppd_mean": ppd_means,
                   "comp_mean": comp_means,
                   "p_val": p_vals}

    sorted_df = pd.DataFrame(ppd_df_dict).sort_values("p_val")
    array_count = np.flip(np.array(range(1, sorted_df.shape[0] + 1)))
    sorted_df["adj_p_val"] = 0.05 / array_count
    sorted_df["HB_corr"] = sorted_df["p_val"] < sorted_df["adj_p_val"]
    
    return sorted_df

# These helper functions are for illustration

def make_sample(df, q, rm_ppd = False):
    """ Takes the positive/negative proportions from a given data frame and 
    turns them into an array.
    
    df -- DataFrame to take proportions from
    q  -- Question to take proportions from"""
    
    ans_count = int(df[df.QUESTION == q].ANSCOUNT)
    
    # Condition to deal with Yes/No answers. Note, one implication of this fix
    # is that 'Agree' is mapped to positive which should be kept in mind when
    # reading questions like Q53
    if q in ["Q28", "Q53", "Q85", "Q87", "Q88", "Q89", "Q90"]:
        no_pos = int(((df[df.QUESTION == q].AGREE / 100) * 
                      ans_count).round())
        no_neg = ans_count - no_pos
    elif q == "Q54":
        print("Q54 is a special case, treat it manually.")
        return np.zeros(ans_count)
    else:
        no_pos = int(((df[df.QUESTION == q].MOST_POSITIVE_OR_LEAST_NEGATIVE / 100) * 
                      ans_count).round())
        no_neg = int(((df[df.QUESTION == q].MOST_NEGATIVE_OR_LEAST_POSITIVE / 100) * 
                      ans_count).round())
        
    if rm_ppd:
        prop_pos, _, ppd_n = calc_mean_se(ppd, q)
        ppd_pos = round(prop_pos * ppd_n)
        ppd_neg = ppd_n - ppd_pos
        
        no_pos -= ppd_pos
        no_neg -= ppd_neg
        ans_count -= ppd_n
    
    sample = np.zeros(ans_count)
    sample[:no_pos] = 1
    
    return sample

def ttest_means(df1, df2, q, equal = False, subset = True):
    """ Generates and applies a t-test to see if the means between two samples
    are statistically different. Returns a tuple with two means and p-value.
    
    df1   -- DataFrame for first sample
    df2   -- DataFrame for second sample
    q     -- Question to test samples for
    equal -- Argument for t-test, default is to assume variance is not equal
    
    Note on equality: in our application equal should = True but this is the
    safer default"""
    
    sample1 = make_sample(df1, q)
    sample2 = make_sample(df2, q, rm_ppd = subset)
    
    if not(sum(sample1) or sum(sample2)):
        return (0, 0, 1)
    
    result = stats.ttest_ind(sample1, sample2, equal_var = equal)
    
    mean1 = sample1.mean() * 100
    mean2 = sample2.mean() * 100
    
    return (mean1, mean2, result[1])

# Note: Permutation test should have a seed set at the start of the file and
# may run slowly with default values

def perm_test(df1, df2, q, boot = 30000, subset = True):
    """ Runs a permutation test to calculate difference between means when
    variance is unknown. Returns a tuple with two means and a p-value.
    
    df1  -- DataFrame for first sample
    df2  -- DataFrame for second sample
    q    -- Question to test samples for
    boot -- Number of runs to perform. Default is 30,000"""
    
    sample1 = make_sample(df1, q)
    sample2 = make_sample(df2, q, rm_ppd = subset)
    abs_mean_diff = abs(sample1.mean() - sample2.mean())
    all_data = np.hstack([sample1, sample2])
    extreme_vals = 0 # Initialize extreme values
    
    for _ in range(boot):
        np.random.shuffle(all_data)
        # Create two random samples drawn from all data
        sample_a = all_data[:sample1.size]
        sample_b = all_data[sample1.size:]
        if abs(sample_a.mean() - sample_b.mean()) >= abs_mean_diff:
            extreme_vals += 1
            
    # p-value is the proportion of extreme observations
    p_value = extreme_vals / boot
        
    mean1 = sample1.mean() * 100
    mean2 = sample2.mean() * 100
    
    return (mean1, mean2, p_value)

## Analysis

The main question we've been asked is how we can improve. A natural starting place is to see where PPD rates lower than its comparison. The next five expressions run the comparisons from the functions above. We can compare the means to see which questions PPD rates lower on, but the result would leave us with over 70 questions to examine. Most of these are likely minor and so we may be more interested in seeing statistically different results. We'll start by comparing the questions for which PPD rates lower.

In [4]:
ppd_statcan = make_comparisons(ppd, statcan)
ppd_econ_stat = make_comparisons(ppd, econ_stat)
ppd_econ_wide_stat = make_comparisons(ppd, econ_wide_stat)
ppd_ps = make_comparisons(ppd, public_service)
ppd_cpd = make_comparisons(ppd[ppd.QUESTION.isin(set(cpd.QUESTION))], cpd, subset = False)

Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.
Q54 is a special case, treat it manually.


In [5]:
statcan_lower = set(ppd_statcan.loc[ppd_statcan.ppd_mean < 
                                    ppd_statcan.comp_mean, "question"])
econ_stat_lower = set(ppd_econ_stat.loc[ppd_econ_stat.ppd_mean < 
                                    ppd_econ_stat.comp_mean, "question"])
econ_wide_stat_lower = set(ppd_econ_wide_stat.loc
                           [ppd_econ_wide_stat.ppd_mean <
                            ppd_econ_wide_stat.comp_mean, "question"])
ps_lower = set(ppd_ps.loc[ppd_ps.ppd_mean < 
                          ppd_ps.comp_mean, "question"])
cpd_lower = set(ppd_cpd.loc[ppd_cpd.ppd_mean <
                            ppd_cpd.comp_mean, "question"])

ppd_statcan = make_comparisons(ppd[ppd.QUESTION.isin(statcan_lower)], statcan)
ppd_econ_stat = make_comparisons(ppd[ppd.QUESTION.isin(econ_stat_lower)],
                                 econ_stat)
ppd_econ_wide_stat = make_comparisons(
    ppd[ppd.QUESTION.isin(econ_wide_stat_lower)], econ_wide_stat)
ppd_ps = make_comparisons(ppd[ppd.QUESTION.isin(ps_lower)], public_service)
ppd_cpd = make_comparisons(ppd[ppd.QUESTION.isin(set(cpd_lower))], cpd, subset = False)

In [6]:
set(ppd_statcan.loc[ppd_statcan["p_val"] < 0.05, "question"])

{'Q18h', 'Q70p', 'Q70x'}

In [7]:
set(ppd_econ_stat.loc[ppd_econ_stat["p_val"] < 0.05, "question"])

{'Q18h', 'Q28', 'Q70b', 'Q70p', 'Q70w', 'Q70x', 'Q93'}

In [8]:
set(ppd_econ_wide_stat.loc[ppd_econ_wide_stat["p_val"] < 0.05, "question"])

{'Q28', 'Q70b', 'Q70x', 'Q93'}

In [9]:
set(ppd_ps.loc[ppd_ps["p_val"] < 0.05, "question"])

{'Q18h', 'Q70p', 'Q70x'}

In [10]:
set(ppd_cpd.loc[ppd_cpd["p_val"] < 0.05, "question"])

{'Q70b', 'Q70c', 'Q70w', 'Q93'}

The results are above but we will summarize them here since the code may not be immediately intuitive:

At a 5% significance level PPD lags behind each comparison in the following questions:
* Public Service: Q18h, Q70p, Q70x
* Statistics Canada: Q18h, Q70p, Q70x
* Economic Statistics: Q18h, Q28, Q70b, Q70p, Q70w, Q70x, Q93
* Economy Wide Statistics: Q28, Q70b, Q70x, Q93
* Consumer Prices : Q70b, Q70c, Q70w, Q93

Some common patterns have emerged among the categories:

* Q18h - "My work suffers because of unreliable technology"
* Q70b - "Overall to waht extent do the following factors cause you stress at work? Pay or other compensation-related issues
* Q70p - "Overall to what extent do the following factors cause you stress at work? Difficulty accessing my work tools or network"
* Q70x - "Overall to what extend to the following factors cause you stress at work? Personal issues"

Broadly speaking, compared to the field level and above PPD appears to lag in terms of access to technology and the stress resulting from it. At lower levels (branch and comparable division) comparisons change in terms of factors contributing to stress with a focus on compensation, including Phoenix related issues.

The difficulty here is that we are testing potentially 216 different questions for differences between means. At a 5% significance level it is entirely possible (even likely) there's at least one Type I error. We may feel more secure in the consistency between the results (technology is cited as a difficulty and also emerges as a source of stress). We do not need to rely on judgement to make this determination but can apply a correction. The make_comparisons function has already applied the Holm-Bonferroni correction (applying a stricter significance level based on the number of comparisons being made).

The results are below:

In [11]:
set(ppd_statcan.loc[ppd_statcan["HB_corr"], "question"])

set()

In [12]:
set(ppd_econ_stat.loc[ppd_econ_stat["HB_corr"], "question"])

set()

In [13]:
set(ppd_econ_wide_stat.loc[ppd_econ_wide_stat["HB_corr"], "question"])

set()

In [14]:
set(ppd_ps.loc[ppd_ps["HB_corr"], "question"])

{'Q18h'}

In [15]:
set(ppd_cpd.loc[ppd_cpd["HB_corr"], "question"])

set()

The only significant difference after the Holm-Bonferroni correction is a comparison between PPD and the public service in terms of unreliable technology. It is worth noting that the Public Service subset has the smallest number of questions (37) to test the hypothesis against and so these results are almost certainly driven by the number of questions being tested. As such, we do not need to abandon the conclusions that come from the hypothesis tests above but these conclusions are not immune to concerns about being the result of mining.

The distinction is worth making when considering the comparisons that are being made. The Holm-Bonferroni correction is, ultimately, an adjustement to the significance level we tolerate and that significance level is itself a judgement call. As mentioned above, if the concern is that the significant results are the products of random chance, we can feel safer in the fact that there is a logical consistency between the technology questions.

Another potential set of internally consistent results are the compensation related questions. Q93 is the question related to Phoenix (against which PPD falls behind Economic Statistics and Consumer Prices at a statistically significant level, but not against Statistics Canada or the Public Service at large) and Q70b list pay and compensation related issues as a source of stress. Unfortunately the remaining remaining questions do not lend themselves to this kind of internal validation. Q70w (lack of job security with unfavourable comparisons against Economic Statistics and Consumer Prices (CPD)), Q70b (pay or other compensation related issues, unfavourable comparison to CPD), and Q70c (heavy workload, unfavourable comparison against CPD) are all relevant but may or may not touch on related issues (e.g. overworked employees may feel underpaid, but overwork may also stem from access to technology).

Why compare with CPD? While the two divisions are separate for a reason, it offers the most natural comparison in terms of the type of work that will be done. For instance, when comparing to Economic Statistics the nature of the work in the other divisions may mean that concerns about technology reflect PPD's different technological needs and the fact that they are difficult to address. If this difference persists between PPD and CPD then it invites the question as to why two divisions doing similar work face such different technological outcomes and recommends a different solution (either learning from what CPD has done to mitigate technological issues, or identify the pain points at PPD that are leading us to suffer more technological difficulties).

An alternative perspective is to see the other comparisons as increasingly granular looks at PPD and its organization parents (which lends itself to the interpretation that differences are due to factors affecting the organizational unit. e.g. PPD is affected by technological outages more than its peers at StatCan), while the CPD comparison looks at similar work (which lends itself to an interpretation as to why there are better or worse outcomes for similar work. e.g. Why does PPD feel overworked compared to CPD?)

## Summary

These results are an extract from a larger analysis of the PSES but were the most likely to enhance the work done by the existing team. The results can be summarized by organizational level.

### Comparisons at the Field Level and Higher
Compared to Statistics Canada and the Public Service as a whole, PPD has a statistically significant gap in questions related to technology, specifically questions Q18h and Q70p (work suffering because of unreliable technology and difficulty accessing work tools being a source of stress). This gap ranged from 27 to 18% fewer positive responses (though the work stress technology question was not significant for the field). In addition, PPD showed a smaller but still statistically significant gap in responses related to personal issues being a source of stress at work (Q70x). While we cannot rule out the possibility that this is a statistical aberration, it remains the most natural place outside of technology to examine due to its large magnitude.

Compared to the Economic Statistics Field PPD also expressed greater concern regarding its job security. While it should be noted that the field as a whole is quite high (95% positive compared to PPD's 83%), a 12 point gap regarding job security is notable and is a clear issue that can be examined and addressed (as opposed to the more intangible personal issues).

### Comparisons at the Branch Level
Once checking below the Field level the technology gap gives way to other concerns. Pay and compensation related issues and personal issues were major sources of stress. These may be related to the Phoneix issues (Q93), but there is wider room for interpretation between a combination of Q70b (pay related stress) and Q93 (Phoenix issues), than there is the technology questions (Q18h and Q70p).

### Comparisons to CPD
PPD was compared to CPD given the potential similarity to the nature of the work. When comparing to CPD the significant differences regarding the technology questions (Q18h and Q70p) were not present (PPD's mean was lower, but by a small magnitude). Notably the significant differences related to job security (Q70w) and Phoenix (Q93) persist. While Phoneix issues are not likely to be related to the nature of the work, the difference in stresses related to job security between the field and a comparable division invites further scrutiny. In addition, heavy workload as a source of stress emerges as a factor against which PPD does not favourably compare.

### Conclusions
The absence of technological differences between the two prices divisions may suggest that these divisions have more demanding technological needs that are not being met by the existing divisions (or, alternatively, are not organized in a way that is harmonious with the delivery of technological services). Regardless of the source, technology affecting the quality of the work and becoming a source of stress is one of the most consistent results from the PSES.

Beyond technological issues issues, PPD appears to be more heavily affected by Phoneix pay issues and concerns about job security at the branch (and lower) level. While we do not find evidence that PPD is any different at the field level or above, the persistence of the gap when comparing against the branch and anotehr division recommends further investigation.